<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=400/>
    <b style="font-size: 30px">Python. Web, HTTP, API</b>
    <br/>
    <br/>
<b style="font-size: 20px">Комендантян Андрей</b>
</center>

# Устройство компьютерных сетей

- Обмен данными по сети устроен многоуровнево – от тока в проводах до картинки в браузере
- Для того что бы использовать сеть, как правило, не требуется хорошо разбираться во всех уровнях

# Сетевые модели: OSI vs TCP/IP

1. OSI эталонная модель
2. TCP/IP более практическая

![osivstcpip](https://zametkinapolyah.ru/wp-content/uploads/2018/05/03-%D0%A1%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5-%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B5%D0%B9-OSI-7-%D0%B8-TCP-IP.png)

# Сеть на низком уровне

<div align="center"><img src="https://www.cisco.com/c/dam/en/us/support/docs/switches/catalyst-3750-series-switches/91672-catl3-wol-vlans1.gif" width="350px" /></div>

Если хочется разобраться, то ["Сети для самых маленьких"](https://linkmeup.gitbook.io/sdsm/) | [[оно же на habr]](https://habr.com/ru/post/134892/)

## Ethernet, IP, TCP и все-все-все

![tt](./TCP_IP.svg)

## IPv4 vs IPv6

![ip](https://i.ytimg.com/vi/D1avGPpSJHs/maxresdefault.jpg)

# TCP vs UDP
- UDP просто посылает сообщения и не заботится о надёжности доставки или порядке прихода сообщений
- TCP выполняет "рукопожатие" между клиентами и может гарантировать надёжность доставки и порядок прихода сообщений

![TCPvsUDP](https://wiki.merionet.ru/images/tcp-i-udp-v-chem-raznica/1.png)

## Порты
- 16-битное число, позволяющее идентифицировать конкретный процесс на машине с одним и тем же IP-адресом

<div align="center"><img src="TCP.png" width="450px" /></div>

# DNS

![](basics-of-dns-resolving-with-cache.jpg)

In [285]:
import socket

In [286]:
socket.gethostbyname('ya.ru')

'87.250.250.242'

# URL: Uniform Resource Locator

![url](http1-url-structure.png)

# HTTP: HyperText Transfer Protocol 
- протокол прикладного уровня (поверх TCP) на базе которого работают многие приложения

## HTTP-request

![request](HTTP_Request.png)

## HTTP methods

![http-methods](HTTP_methods.jpg)

## HTTP Headers

![request-headers](HTTP_Request_Headers2.png)

[Mozilla docs: list of HTTP headers](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers)

## HTTP-response

![response](HTTP_Response.png)

<div align="center"><img src="HTTP_statuses.jpeg" width="700px" /></div>

[Mozilla docs: list of HTTP statuses](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status)

## HTTP response

![response-headers](HTTP_Response_Headers2.png)

## send via UDP 

In [289]:
addr = '127.0.0.1', 6001  # host, port

In [288]:
sock_one = socket.socket(
    socket.AF_INET,  # Address Family _ INET
    socket.SOCK_DGRAM  # ≈ "use UDP"
)
sock_one.bind(addr)

with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock_two:
    sock_two.sendto(b'Hello,', addr)
    sock_two.sendto(b'world!', addr)
    
with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock_three:
    sock_three.sendto(b'How', addr)
    sock_three.sendto(b'are', addr)
    sock_three.sendto(b'you?', addr)

for _ in range(5):
    data, addr = sock_one.recvfrom(1024)
    print(data, addr)

sock_one.close()

b'Hello,' ('127.0.0.1', 38717)
b'world!' ('127.0.0.1', 38717)
b'How' ('127.0.0.1', 58153)
b'are' ('127.0.0.1', 58153)
b'you?' ('127.0.0.1', 58153)


## send via TCP

In [297]:
addr = '87.250.250.242', 80  # host, port (standart port for HTTP)

In [298]:
request = b'\r\n'.join([
    b'GET / HTTP/1.1',
    # b'Host: ya.ru',
]) + b'\r\n\r\n'

with socket.socket(
    family=socket.AF_INET,  # Adress Family _ INET
    type=socket.SOCK_STREAM  # ≈ "use TCP"
) as sock:
    sock.connect(addr)
    sock.sendall(request)
    response = sock.recv(1024)

In [299]:
print(response.decode('latin1'))

HTTP/1.1 406 Not acceptable
Connection: Close
Content-Length: 0




# SSL / TLS

<div align="center"><img src="what-is-ssl-handshake.svg" width="700px" /></div>

In [117]:
import ssl
from functools import partial

In [118]:
context = ssl.create_default_context()
saddr = addr[0], 443  # HTTPS standart port

In [119]:
chunks = []

with socket.create_connection(saddr) as sock:
    with context.wrap_socket(sock, server_hostname='ya.ru') as ssock:
        ssock.sendall(request)
        for chunk in iter(partial(ssock.recv, 2048), b''):
            chunks.append(chunk)

response = b''.join(chunks)

In [120]:
headers, data = response.split(b'\r\n\r\n')

[Больше о работе с сокетами в питоне](https://docs.python.org/3/howto/sockets.html)

In [81]:
print(headers.decode('latin1'))

HTTP/1.1 200 Ok
Accept-CH: Viewport-Width, DPR, Device-Memory, RTT, Downlink, ECT
Accept-CH-Lifetime: 31536000
Cache-Control: no-cache,no-store,max-age=0,must-revalidate
Connection: Close
Content-Length: 60503
Content-Security-Policy: connect-src https://*.mc.yandex.ru https://adstat.yandex.ru https://mc.admetrica.ru https://mc.yandex.ru https://yandex.ru;default-src 'none';frame-src 'self' https://mc.yandex.md https://mc.yandex.ru;img-src 'self' data: https://*.mc.yandex.ru https://*.verify.yandex.ru https://adstat.yandex.ru https://avatars.mds.yandex.net https://favicon.yandex.net https://mc.admetrica.ru https://mc.yandex.com https://mc.yandex.ru https://yandex.ru https://yastatic.net;report-uri https://csp.yandex.net/csp?project=morda&from=morda.yaru.ru&showid=1637625094.66897.99897.105126&h=stable-morda-yaru-man-yp-9&yandexuid=4033075521637625094;script-src 'unsafe-inline' https://*.mc.yandex.ru https://adstat.yandex.ru https://mc.yandex.ru https://yandex.ru https://yastatic.net;st

In [82]:
html = data.decode('utf-8')

In [83]:
html[:512]

'<!DOCTYPE html><html   class="i-ua_js_no i-ua_css_standart i-ua_browser_ i-ua_browser-engine_ i-ua_browser_desktop i-ua_platform_other"  lang="ru"><head  xmlns:og="http://ogp.me/ns#"><meta http-equiv=\'Content-Type\' content=\'text/html;charset=UTF-8\'><title>Яндекс</title><link rel="preload" href="//yastatic.net/jquery/1.8.3/jquery.min.js" as="script" crossorigin="anonymous"><link rel="preload" href="//yastatic.net/s3/home-static/_/K/k/uSVxlDs06YE3bN4Vn8mwL3WHk.js" as="script" crossorigin="anonymous"><link rel'

# Requests: HTTP for Humans
- удобная библиотека для работы с HTTP в python

In [124]:
import requests

In [125]:
response = requests.get('http://ya.ru')

In [126]:
response.status_code

200

In [127]:
response.text[:512]

'<!DOCTYPE html><html   class="i-ua_js_no i-ua_css_standart i-ua_browser_unknown i-ua_browser-engine_unknown i-ua_browser_desktop i-ua_platform_other"  lang="ru"><head  xmlns:og="http://ogp.me/ns#"><meta http-equiv=\'Content-Type\' content=\'text/html;charset=UTF-8\'><title>Яндекс</title><link rel="preload" href="//yastatic.net/jquery/1.8.3/jquery.min.js" as="script" crossorigin="anonymous"><link rel="preload" href="//yastatic.net/s3/home-static/_/K/k/uSVxlDs06YE3bN4Vn8mwL3WHk.js" as="script" crossorigin="anonym'

In [128]:
html = response.text

# HTML: HyperText Markup Language

![html](https://blogwork.ru/wp-content/uploads/2016/08/blogwork-HTML-tag-p-for-Beginner.png)

![html+css+js](https://miro.medium.com/max/3328/1*l4xICbIIYlz1OTymWCoUTw.jpeg)

In [129]:
from bs4 import BeautifulSoup

In [149]:
html = requests.get('http://example.com').text

In [174]:
soup = BeautifulSoup(html)

In [175]:
css_element = soup.html.head.style
print(css_element)
css_element.clear()

<style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>


In [176]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Example Domain
  </title>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <style type="text/css">
  </style>
 </head>
 <body>
  <div>
   <h1>
    Example Domain
   </h1>
   <p>
    This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.
   </p>
   <p>
    <a href="https://www.iana.org/domains/example">
     More information...
    </a>
   </p>
  </div>
 </body>
</html>



In [177]:
html = soup.html

In [178]:
html.head.title

<title>Example Domain</title>

In [179]:
html.head.findAll('meta')

[<meta charset="utf-8"/>,
 <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>]

In [180]:
paragraphs = html.findAll('p')
paragraphs

[<p>This domain is for use in illustrative examples in documents. You may use this
     domain in literature without prior coordination or asking for permission.</p>,
 <p><a href="https://www.iana.org/domains/example">More information...</a></p>]

In [181]:
links = html.findAll('a')
links

[<a href="https://www.iana.org/domains/example">More information...</a>]

In [182]:
for link in links:
    print(link.text, link['href'])

More information... https://www.iana.org/domains/example


In [302]:
#  https://www.google.ru/search?q=test
SEARCH_URL = 'https://www.google.ru/search'
response = requests.get(SEARCH_URL, params={'q': 'test'})
response.raise_for_status()  # requests умеет сам обрабатывать HTTP-ошибки и бросасть исключения, если всё плохо

In [303]:
soup = BeautifulSoup(response.text)  

In [304]:
for elem in soup.find_all('div', attrs={'class': 'ZINbbc'}):
    descr = elem.find('div', attrs={'class': 'BNeawe'})
    if not descr:
        continue
    print(descr.text, link['href'])

Speedtest от Ookla - Глобальный тест скорости ... https://www.idrlabs.com/tests.php
Test IT https://www.idrlabs.com/tests.php
Тесты на коронавирус и иммунитет бесплатно в Москве https://www.idrlabs.com/tests.php
Тест - Википедия https://www.idrlabs.com/tests.php
test - Википедия https://www.idrlabs.com/tests.php
Test My Site - Think with Google https://www.idrlabs.com/tests.php
Let's test: Онлайн система тестирования и конструктор тестов https://www.idrlabs.com/tests.php
Speedtest - тест скорости интернета https://www.idrlabs.com/tests.php
Fast.com: Internet Speed Test https://www.idrlabs.com/tests.php
Тест скорости печати онлайн - Ratatype https://www.idrlabs.com/tests.php
Похожие запросы https://www.idrlabs.com/tests.php


In [305]:
SEARCH_URL = 'https://duckduckgo.com/html'  # версия сайта без /html работает только с JavaScript
                                            # распарсить google.com не получилось...
response = requests.get(
    "https://duckduckgo.com/html", params={'q': 'test'},
    headers={'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                           'AppleWebKit/537.36 (KHTML, like Gecko) '
                           'Chrome/77.0.3865.121 '
                           'Safari/537.36'}
)
response.raise_for_status()

In [227]:
soup = BeautifulSoup(response.text)

In [306]:
from IPython.display import display, HTML

In [238]:
for link in soup.find_all('a', attrs={"class": "result__a"}):
    display(HTML(str(link)))

In [236]:
for div in soup.find_all('div', attrs={"class": "results_links"}):
    display(HTML(str(div)))

# API: Application Programming Interface

![api](http://blog.restcase.com/content/images/2016/11/api-collaboration.png)

## Публичное API

In [307]:
DICTIONARY_API = 'https://dictionary.yandex.net/api/v1/dicservice.json/lookup'
TEMP_KEY = 'dict.1.1.20191119T132629Z.f06d32fa3b4c5e18.922a67da8890eb40fb4a680a7d5d10347f8139b7'

In [308]:
params = {'key': TEMP_KEY, 'lang': 'en-ru', 'text': 'hello'}
response = requests.get(DICTIONARY_API, params=params)
response.raise_for_status()
body = response.text

In [309]:
body

'{"head":{},"def":[{"text":"hello","pos":"noun","ts":"həˈləʊ","tr":[{"text":"привет","pos":"noun","gen":"м","fr":10,"syn":[{"text":"добрый день","pos":"noun","fr":5},{"text":"Здравствуй","pos":"noun","gen":"м","fr":1}],"mean":[{"text":"hi"},{"text":"good afternoon"}],"ex":[{"text":"big hello","tr":[{"text":"большой привет"}]}]},{"text":"приветствие","pos":"noun","gen":"ср","fr":5}]},{"text":"hello","pos":"verb","ts":"həˈləʊ","tr":[{"text":"здравствуйте","pos":"verb","asp":"несов","fr":1,"mean":[{"text":"hi"}]},{"text":"поздороваться","pos":"verb","asp":"сов","fr":5,"mean":[{"text":"greet"}]},{"text":"приветствовать","pos":"verb","asp":"несов","fr":5}]},{"text":"hello","pos":"interjection","ts":"həˈləʊ","tr":[{"text":"АЛЛО","pos":"interjection","fr":10},{"text":"ау","pos":"interjection","fr":1}]},{"text":"hello","pos":"adverb","ts":"həˈləʊ","tr":[{"text":"здорово","pos":"adverb","fr":1,"mean":[{"text":"hey"}]}]}]}'

In [310]:
response.headers['content-type']

'application/json; charset=utf-8'

# JSON: JavaScript Object Notation

![json](https://www.w3resource.com/w3r_images/json-introduction.png)

In [311]:
import json

In [312]:
definition = json.loads(body)['def']  # or response.json()['def']

In [313]:
definition[:2]

[{'text': 'hello',
  'pos': 'noun',
  'ts': 'həˈləʊ',
  'tr': [{'text': 'привет',
    'pos': 'noun',
    'gen': 'м',
    'fr': 10,
    'syn': [{'text': 'добрый день', 'pos': 'noun', 'fr': 5},
     {'text': 'Здравствуй', 'pos': 'noun', 'gen': 'м', 'fr': 1}],
    'mean': [{'text': 'hi'}, {'text': 'good afternoon'}],
    'ex': [{'text': 'big hello', 'tr': [{'text': 'большой привет'}]}]},
   {'text': 'приветствие', 'pos': 'noun', 'gen': 'ср', 'fr': 5}]},
 {'text': 'hello',
  'pos': 'verb',
  'ts': 'həˈləʊ',
  'tr': [{'text': 'здравствуйте',
    'pos': 'verb',
    'asp': 'несов',
    'fr': 1,
    'mean': [{'text': 'hi'}]},
   {'text': 'поздороваться',
    'pos': 'verb',
    'asp': 'сов',
    'fr': 5,
    'mean': [{'text': 'greet'}]},
   {'text': 'приветствовать', 'pos': 'verb', 'asp': 'несов', 'fr': 5}]}]

## API с OAuth авторизацией и python-клиент

In [314]:
class YandexLoginClient:
    """Клиент для доступа к общей информации о профиле в Яндексе"""
    
    def __init__(self, *, base_url="https://login.yandex.ru", token):
        self._base_url = base_url
        self._session = requests.Session()
        self._session.headers['Authorization'] = f'OAuth {token}'
    
    def info(self):
        response = self._session.get(self._base_url + '/info')
        response.raise_for_status()
        return response.json()
    
    def censored_email(self):
        email = self.info()['default_email']
        name, host = email.split('@')
        return name[0] + '*' * (len(name) - 1) + '@' + host

In [315]:
from pathlib import Path
token = (Path.home() / '.PythonWebHTTPApiLecture/token').read_text().strip()
client = YandexLoginClient(token=token)

In [316]:
client.censored_email()

'k******@yandex.ru'

<small>OAuth токен можно полуить тут

https://oauth.yandex.ru/authorize?response_type=token&client_id=4bc448e01f46442883dce6dc3254e44c&device_id=my_device

и положить в ~/.PythonWebHTTPApiLecture/token</small>

# Непубличное API

Любые WEB-приложения для обновления данных внутри себя ходят в какие-то внутренние API своей платформы. При некотором старании можно научиться пользоваться этими API в вашем коде. Впрочем, для непубличного API гарантий никаких – сломаться может в любой момент.

# Спасибо за внимание!